In [1]:
# Required imports
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd


In [2]:
#1.Loading Original Dataset : 
dataset=pd.read_csv("Pre-processed_CKD_Data.csv",index_col=None)
print(dataset.shape)
dataset.head()

(399, 25)


,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.0,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.0,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.0,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.0,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.0,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes


In [3]:
#2.Duplicating the Original Dataset
dataset2 = dataset

#3.Classifying the Nominal Columns in Dataset : 
dataset2 = pd.get_dummies(dataset2, drop_first=True)
print(dataset2.shape)
dataset2.head()

(399, 28)


,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.0,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.0,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.0,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.0,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.0,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1


In [4]:
# ✅ 4. Assigning Variables (Independent/Dependent)
indep_X = dataset2.drop('classification_yes', axis=1)
print(indep_X.shape)

dep_Y = dataset2['classification_yes']
print(dep_Y.shape)

(399, 27)
(399,)


In [5]:

# ✅ 5. Created Function - RFE_Features_Classification
def RFE_Features_Classification(indep_X, dep_Y, n):
    RFE_List = []
    model = LogisticRegression(max_iter=1000, solver='lbfgs')  # Solver & iteration fixed
    rfe = RFE(model, n_features_to_select=n)
    rfe_fit = rfe.fit(indep_X, dep_Y)
    selected_features = rfe.transform(indep_X)
    RFE_List.append(selected_features)
    return RFE_List

RFE_List = RFE_Features_Classification(indep_X, dep_Y, 6)

C:\Anaconda\envs\AI_ML\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [6]:
# ✅ 6. Created Function - train_test_split_and_StandardScaler
def train_test_split_and_StandardScaler(indep_X, dep_Y):
    X_train, X_test, Y_train, Y_test = train_test_split(indep_X, dep_Y, test_size=0.3, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, Y_train, Y_test

In [7]:
# ✅ 7 to 13: Classifier Functions (return same format)
def LogisticRegression_Model(X_train, Y_train, X_test):
    model = LogisticRegression(max_iter=1000, solver='lbfgs')
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    return model, X_test, Y_test, confusion_matrix(Y_test, Y_pred), classification_report(Y_test, Y_pred), accuracy_score(Y_test, Y_pred)

def SVM_Linear(X_train, Y_train, X_test):
    model = SVC(kernel='linear', gamma='scale')
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    return model, X_test, Y_test, confusion_matrix(Y_test, Y_pred), classification_report(Y_test, Y_pred), accuracy_score(Y_test, Y_pred)

def SVM_Non_Linear(X_train, Y_train, X_test):
    model = SVC(kernel='rbf', gamma='scale')
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    return model, X_test, Y_test, confusion_matrix(Y_test, Y_pred), classification_report(Y_test, Y_pred), accuracy_score(Y_test, Y_pred)

def KNN(X_train, Y_train, X_test):
    model = KNeighborsClassifier()
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    return model, X_test, Y_test, confusion_matrix(Y_test, Y_pred), classification_report(Y_test, Y_pred), accuracy_score(Y_test, Y_pred)

def Naive_Bayes(X_train, Y_train, X_test):
    model = GaussianNB()
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    return model, X_test, Y_test, confusion_matrix(Y_test, Y_pred), classification_report(Y_test, Y_pred), accuracy_score(Y_test, Y_pred)

def DecisionTree(X_train, Y_train, X_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    return model, X_test, Y_test, confusion_matrix(Y_test, Y_pred), classification_report(Y_test, Y_pred), accuracy_score(Y_test, Y_pred)

def RandomForest(X_train, Y_train, X_test):
    model = RandomForestClassifier()
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    return model, X_test, Y_test, confusion_matrix(Y_test, Y_pred), classification_report(Y_test, Y_pred), accuracy_score(Y_test, Y_pred)

In [8]:
# ✅ 14. Created Function - RFE_Classification (to summarize accuracy)
def RFE_Classification(acc_lr, acc_svm_linear, acc_svm_nonlinear, acc_knn, acc_nb, acc_dt, acc_rf):
    df = pd.DataFrame({
        'LogisticRegression': acc_lr,
        'SVM_Linear': acc_svm_linear,
        'SVM_NonLinear': acc_svm_nonlinear,
        'KNN': acc_knn,
        'NaiveBayes': acc_nb,
        'DecisionTree': acc_dt,
        'RandomForest': acc_rf
    })
    return df


In [9]:
# ✅ Empty lists to collect accuracy scores
accuracy_LogisticRegression = []
accuracy_SVM_Linear = []
accuracy_SVM_NonLinear = []
accuracy_KNN = []
accuracy_NaiveBayes = []
accuracy_DecisionTree = []
accuracy_RandomForest = []


In [10]:
# ✅ Main Loop
for i in RFE_List:
    X_train, X_test, Y_train, Y_test = train_test_split_and_StandardScaler(i, dep_Y)

    classifier, X_test, Y_test, cm, cr, acc = LogisticRegression_Model(X_train, Y_train, X_test)
    accuracy_LogisticRegression.append(acc)

    classifier, X_test, Y_test, cm, cr, acc = SVM_Linear(X_train, Y_train, X_test)
    accuracy_SVM_Linear.append(acc)

    classifier, X_test, Y_test, cm, cr, acc = SVM_Non_Linear(X_train, Y_train, X_test)
    accuracy_SVM_NonLinear.append(acc)

    classifier, X_test, Y_test, cm, cr, acc = KNN(X_train, Y_train, X_test)
    accuracy_KNN.append(acc)

    classifier, X_test, Y_test, cm, cr, acc = Naive_Bayes(X_train, Y_train, X_test)
    accuracy_NaiveBayes.append(acc)

    classifier, X_test, Y_test, cm, cr, acc = DecisionTree(X_train, Y_train, X_test)
    accuracy_DecisionTree.append(acc)

    classifier, X_test, Y_test, cm, cr, acc = RandomForest(X_train, Y_train, X_test)
    accuracy_RandomForest.append(acc)

In [11]:
# ✅ Final Summary
result = RFE_Classification(
    accuracy_LogisticRegression,
    accuracy_SVM_Linear,
    accuracy_SVM_NonLinear,
    accuracy_KNN,
    accuracy_NaiveBayes,
    accuracy_DecisionTree,
    accuracy_RandomForest
)

result

,LogisticRegression,SVM_Linear,SVM_NonLinear,KNN,NaiveBayes,DecisionTree,RandomForest
0,0.966667,0.958333,0.958333,0.966667,0.958333,0.966667,0.966667


In [12]:
# Calling the Final Results by Selecting Top 6 Features
#result

In [13]:
# Calling the Final Results by Selecting Top 5 Features
#result

In [14]:
# Calling the Final Results by Selecting Top 4 Features
#result

In [15]:
# Calling the Final Results by Selecting Top 3 Features
#result

In [16]:
# Calling the Final Results by Selecting Top 2 Features
#result

In [17]:
# Calling the Final Results by Selecting Top 1 Feature
#result